In [7]:
from PIL import Image 
import glob as gb
import numpy as np
import keras


In [8]:
image_width  = 40
image_height = 40

In [9]:
def load_hoda_image(path):
    all_files = gb.glob( path + "/*.bmp")
    data = []
    for i in range(len(all_files)):
        image_file = Image.open(all_files[i]) # open colour image
        image_file = image_file.convert('L')
        image_file = image_file.resize((image_width, image_height), Image.ANTIALIAS)
        np_array_file = np.array(image_file)
        data.append(np_array_file)
    return np.array(data)

In [10]:
def load_hoda_labels(path):
    all_files = gb.glob( path + "/*.bmp")
    data = []
    for i in range(len(all_files)):
        x = all_files[i].split("_")
        y = x[1].replace(".bmp" , "")
        data.append(y)
    return np.array(data)

In [11]:
def load_hoda():
    test_images = load_hoda_image('datasets/hoda/test')
    test_labels = load_hoda_labels('datasets/hoda/test')
    train_images = load_hoda_image('datasets/hoda/train')
    train_labels = load_hoda_labels('datasets/hoda/train')
    return train_images, train_labels, test_images, test_labels
    

In [12]:
train_images, train_labels, test_images, test_labels = load_hoda()

In [13]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(image_width * image_height,)))
#network.add(layers.Dense(512, activation='relu'))

network.add(layers.Dense(10, activation='softmax'))

In [14]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [15]:
train_images = train_images.reshape((60000, image_width * image_height))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((20000, image_width * image_height))
test_images = test_images.astype('float32') / 255

In [16]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [17]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 9s 155us/step - loss: 0.1544 - acc: 0.9567
Epoch 2/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0508 - acc: 0.9852
Epoch 3/5
60000/60000 [==============================] - 9s 150us/step - loss: 0.0354 - acc: 0.9896
Epoch 4/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.0275 - acc: 0.9916
Epoch 5/5
60000/60000 [==============================] - 9s 150us/step - loss: 0.0239 - acc: 0.9932


In [18]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

20000/20000 [==============================] - 1s 67us/step


In [19]:
print('test_acc:', test_acc)

test_acc: 0.982


In [20]:
def one_hot_encode_to_number(one_hot_coded):
    val = 0
    for i in range(0,10,1):
        val += i * one_hot_coded[i]
    return val

In [21]:
def one_hot_encode_percent_number(one_hot_coded):
    result = [];
    for i in range(0,10,1):
        per = one_hot_coded[i] * 100
        result.append(round(per,3))
    return result

In [22]:
def predicate_result(network, index):
    inp = np.array([test_images[index]])
    real_label = one_hot_encode_to_number(test_labels[index])
    result = one_hot_encode_percent_number(network.predict(inp)[0])
    return real_label, result

In [23]:
real_label, percent_result = predicate_result(network, 120)
print('real label is ', real_label)
for i in range(len(percent_result)):
    print(i, ':' , '{:6}%'.format(percent_result[i]))


real label is  0.0
0 :  100.0%
1 :    0.0%
2 :    0.0%
3 :    0.0%
4 :    0.0%
5 :    0.0%
6 :    0.0%
7 :    0.0%
8 :    0.0%
9 :    0.0%
